Author: Eric Ballam

Date: 11/24/2021

Resources: https://data.cdc.gov/NCHS/Indicators-of-Reduced-Access-to-Care-Due-to-the-Co/xb3p-q62w

### Overview
The data from the US Census Bureau contains information on the impact covid had on differnet groups access to medical care. The purpose of this work is to identify which groups were most significatly impacted by covid and to model when access to care will return to normal. 

The data for this project was orginally collected in March 2021 with data collected again in June to check model accuracy. The first step in this process is to understand the features of the data and decided how best to use it.  

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import datetime

In [2]:
month = 'MAR'
df = pd.read_csv(f"DATA/{month} - Indicators_of_Reduced_Access_to_Care_Due_to_the_Coronavirus_Pandemic_During_Last_4_Weeks.csv")
df.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,"Delayed Medical Care, Last 4 Weeks",National Estimate,United States,United States,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,38.7,38.1,39.4,38.1 - 39.4,NaN
1,"Delayed Medical Care, Last 4 Weeks",By Age,United States,18 - 29 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,34.8,32.5,37.2,32.5 - 37.2,NaN
2,"Delayed Medical Care, Last 4 Weeks",By Age,United States,30 - 39 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,37.3,35.6,39.0,35.6 - 39.0,NaN
3,"Delayed Medical Care, Last 4 Weeks",By Age,United States,40 - 49 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,40.3,39.0,41.7,39.0 - 41.7,NaN
4,"Delayed Medical Care, Last 4 Weeks",By Age,United States,50 - 59 years,1,1,Apr 23 - May 5,04/23/2020 12:00:00 AM,05/05/2020 12:00:00 AM,40.7,39.3,42.2,39.3 - 42.2,NaN


In [3]:
# A quick look into dataframe info shows a number of missing values in the numeric columns and a significat issue with the Quartile Range

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5154 entries, 0 to 5153
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               5154 non-null   object 
 1   Group                   5154 non-null   object 
 2   State                   5154 non-null   object 
 3   Subgroup                5154 non-null   object 
 4   Phase                   5154 non-null   object 
 5   Time Period             5154 non-null   int64  
 6   Time Period Label       5154 non-null   object 
 7   Time Period Start Date  5154 non-null   object 
 8   Time Period End Date    5154 non-null   object 
 9   Value                   5040 non-null   float64
 10  Low CI                  5040 non-null   float64
 11  High CI                 5040 non-null   float64
 12  Confidence Interval     5040 non-null   object 
 13  Quartile Range          3672 non-null   object 
dtypes: float64(3), int64(1), object(10)
memo

In [4]:
# This column appears to be a counter for when the data was collected

df['Time Period'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int64)

In [5]:
# This column indicates what kind of delay covid casused, for our analyiss we will use Delayed or Did Not Get Care, Last 4 Weeks

df.Indicator.unique()

array(['Delayed Medical Care, Last 4 Weeks',
       'Did Not Get Needed Care, Last 4 Weeks',
       'Delayed or Did Not Get Care, Last 4 Weeks'], dtype=object)

In [6]:
# This grouping breaks the data up into large segements

df.Group.unique()

array(['National Estimate', 'By Age', 'By Gender',
       'By Race/Hispanic ethnicity', 'By Education', 'By State'],
      dtype=object)

In [7]:
# This grouping breaks the data up into the specific segements, this is what we will end up using for our analysis

df.Subgroup.unique()

array(['United States', '18 - 29 years', '30 - 39 years', '40 - 49 years',
       '50 - 59 years', '60 - 69 years', '70 - 79 years',
       '80 years and above', 'Male', 'Female', 'Hispanic or Latino',
       'Non-Hispanic white, single race',
       'Non-Hispanic black, single race',
       'Non-Hispanic Asian, single race',
       'Non-Hispanic, other races and multiple races',
       'Less than a high school diploma', 'High school diploma or GED',
       "Some college/Associate's degree", "Bachelor's degree or higher",
       'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina

In [8]:
# This column appears to break up the data into geographical areas based on states

df.State.unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [9]:
# The United States value for this column appears to be the US average

df[df.State == 'United States'].Subgroup.unique()

array(['United States', '18 - 29 years', '30 - 39 years', '40 - 49 years',
       '50 - 59 years', '60 - 69 years', '70 - 79 years',
       '80 years and above', 'Male', 'Female', 'Hispanic or Latino',
       'Non-Hispanic white, single race',
       'Non-Hispanic black, single race',
       'Non-Hispanic Asian, single race',
       'Non-Hispanic, other races and multiple races',
       'Less than a high school diploma', 'High school diploma or GED',
       "Some college/Associate's degree", "Bachelor's degree or higher"],
      dtype=object)

In [10]:
# Each state doesn't contain all the values in subgroup, just the State. Making this column not very useful

df[df.State == 'Alabama'].Subgroup.unique()

array(['Alabama'], dtype=object)

In [11]:
# These time stamps are in string format and will need to be changed into datatime objects 

df['Time Period Start Date'].iloc[0]

'04/23/2020 12:00:00 AM'

In [12]:
# Having a state and end data timestamp probably won't be useful. We will probably end up using start date for our analysis 

df['Time Period End Date'].iloc[0]

'05/05/2020 12:00:00 AM'

In [13]:
# Converting the time string to datetime objects 

df.loc[:,'Time Period Start Date'] = df['Time Period Start Date'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%m/%d/%Y').date()) 
df.loc[:,'Time Period End Date'] = df['Time Period End Date'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%m/%d/%Y').date()) 

In [14]:
# Value is the main indicator varable in the data, it has a number of missing values that must be delt with

df[df.Value.isnull()]

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
2520,"Delayed Medical Care, Last 4 Weeks",National Estimate,United States,United States,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2521,"Delayed Medical Care, Last 4 Weeks",By Age,United States,18 - 29 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2522,"Delayed Medical Care, Last 4 Weeks",By Age,United States,30 - 39 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2523,"Delayed Medical Care, Last 4 Weeks",By Age,United States,40 - 49 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
2524,"Delayed Medical Care, Last 4 Weeks",By Age,United States,50 - 59 years,-1,1,July 22 - Aug 18,2020-07-22,2020-08-18,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4659,"Delayed or Did Not Get Care, Last 4 Weeks",By Race/Hispanic ethnicity,United States,"Non-Hispanic, other races and multiple races",-1,1,Dec 22 - Jan 5,2020-12-22,2021-01-05,NaN,NaN,NaN,NaN,NaN
4660,"Delayed or Did Not Get Care, Last 4 Weeks",By Education,United States,Less than a high school diploma,-1,1,Dec 22 - Jan 5,2020-12-22,2021-01-05,NaN,NaN,NaN,NaN,NaN
4661,"Delayed or Did Not Get Care, Last 4 Weeks",By Education,United States,High school diploma or GED,-1,1,Dec 22 - Jan 5,2020-12-22,2021-01-05,NaN,NaN,NaN,NaN,NaN
4662,"Delayed or Did Not Get Care, Last 4 Weeks",By Education,United States,Some college/Associate's degree,-1,1,Dec 22 - Jan 5,2020-12-22,2021-01-05,NaN,NaN,NaN,NaN,NaN


In [15]:
# All the missing values in the Value field are in Phase -1

df[df.Value.isnull()].Phase.unique()

array(['-1'], dtype=object)

In [16]:
# Looking at the info for Phase -1 we can safely removed it witout worring about losing other information

df[df.Phase == '-1'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 2520 to 4663
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               114 non-null    object 
 1   Group                   114 non-null    object 
 2   State                   114 non-null    object 
 3   Subgroup                114 non-null    object 
 4   Phase                   114 non-null    object 
 5   Time Period             114 non-null    int64  
 6   Time Period Label       114 non-null    object 
 7   Time Period Start Date  114 non-null    object 
 8   Time Period End Date    114 non-null    object 
 9   Value                   0 non-null      float64
 10  Low CI                  0 non-null      float64
 11  High CI                 0 non-null      float64
 12  Confidence Interval     0 non-null      object 
 13  Quartile Range          0 non-null      object 
dtypes: float64(3), int64(1), object(10)
me

In [17]:
# Removing Phase -1 also removes nearly all missing values, with the exception of the missing values in Quartile Range

df = df[df.Phase != '-1']
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5040 entries, 0 to 5153
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               5040 non-null   object 
 1   Group                   5040 non-null   object 
 2   State                   5040 non-null   object 
 3   Subgroup                5040 non-null   object 
 4   Phase                   5040 non-null   object 
 5   Time Period             5040 non-null   int64  
 6   Time Period Label       5040 non-null   object 
 7   Time Period Start Date  5040 non-null   object 
 8   Time Period End Date    5040 non-null   object 
 9   Value                   5040 non-null   float64
 10  Low CI                  5040 non-null   float64
 11  High CI                 5040 non-null   float64
 12  Confidence Interval     5040 non-null   object 
 13  Quartile Range          3672 non-null   object 
dtypes: float64(3), int64(1), object(10)
memo

In [18]:
# Investigating the Phase column further we can see their are some strange values

df.Phase.unique()

array(['1', '2', '3 (Oct 28 � Dec 21)', '3 (Jan 6 � Feb 15)'],
      dtype=object)

In [19]:
# Each phase appears to be a set of dates where data was collected

df[df.Phase == '1']['Time Period Start Date'].unique()

array([datetime.date(2020, 4, 23), datetime.date(2020, 5, 7),
       datetime.date(2020, 5, 14), datetime.date(2020, 5, 21),
       datetime.date(2020, 5, 28), datetime.date(2020, 6, 4),
       datetime.date(2020, 6, 11), datetime.date(2020, 6, 18),
       datetime.date(2020, 6, 25), datetime.date(2020, 7, 2),
       datetime.date(2020, 7, 9), datetime.date(2020, 7, 16)],
      dtype=object)

In [20]:
df[df.Phase == '2']['Time Period Start Date'].unique()

array([datetime.date(2020, 8, 19), datetime.date(2020, 9, 2),
       datetime.date(2020, 9, 16), datetime.date(2020, 9, 30),
       datetime.date(2020, 10, 14)], dtype=object)

In [21]:
# While this phase has a strange name it does have dates and data associated with it

df[df.Phase == '3 (Oct 28 � Dec 21)']['Time Period Start Date'].unique()

array([datetime.date(2020, 10, 28), datetime.date(2020, 11, 11),
       datetime.date(2020, 11, 25), datetime.date(2020, 12, 9)],
      dtype=object)

In [22]:
# Replacing 3 (Oct 28 � Dec 21) with just 3

df.loc[df.Phase == '3 (Oct 28 � Dec 21)', 'Phase'] = 3

In [23]:
# While this phase has a strange name it does have dates and data associated with it

df[df.Phase == '3 (Jan 6 � Feb 15)']['Time Period Start Date'].unique()

array([datetime.date(2021, 1, 6), datetime.date(2021, 1, 20),
       datetime.date(2021, 2, 3)], dtype=object)

In [24]:
# Replacing 3 (Oct 28 � Dec 21) with just 3

df.loc[df.Phase == '3 (Jan 6 � Feb 15)', 'Phase'] = 3

In [25]:
# Double checking Phase

df.Phase.unique()

array(['1', '2', 3], dtype=object)

### Final decisions
After looking through the data provided by US Census Bureau we can make some final decisions on what data will be useful for visulization and modeling. 

- Indicator
     - We will only keep the rows labeled **'Delayed or Did Not Get Care, Last 4 Weeks'** for our analysis
     - After selecting for the label this column will be dropped
- Group
    - We will drop this column in favor of Subgroup
- State
    - We will drop this column, as we showed before it doesn't add anything not already in subgroup
- Subgroup
    - This is an important column for both visulizations and modeling
- Phase
    - This column may be used for visulizations and modeling so it will be kept
- Time Period
    - This column will be kept to help with modeling
- Time Period Label
    - This column doens't add any information not included in **Time Period State Date** or **Time Period End Date** and so will be dropped
- Time Period Start Date
    - This is an important date for tracking when each data point was collected
- Time Period End Date
    - We will drop this column and simply use **Time Period State Date** for all our time based indicators
- Value
    - This column contains the actual data collected and will be used for plotting and modeling
- Low CI
    - This column has the lower range for the value and may be useful for plotting and modeling
- High CI
    - This column has the high range for the value and may be useful for plotting and modeling
- Confidence Interval
    - We will drop this column, it doesn't have any information not contained in **Low CI** or **High CI**
- Quartile Range
    - We will drop this column, it is missing a number of values and so we will use the CI values instead of quartiles

In [26]:
df

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,"Delayed Medical Care, Last 4 Weeks",National Estimate,United States,United States,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,38.7,38.1,39.4,38.1 - 39.4,NaN
1,"Delayed Medical Care, Last 4 Weeks",By Age,United States,18 - 29 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,34.8,32.5,37.2,32.5 - 37.2,NaN
2,"Delayed Medical Care, Last 4 Weeks",By Age,United States,30 - 39 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,37.3,35.6,39.0,35.6 - 39.0,NaN
3,"Delayed Medical Care, Last 4 Weeks",By Age,United States,40 - 49 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,40.3,39.0,41.7,39.0 - 41.7,NaN
4,"Delayed Medical Care, Last 4 Weeks",By Age,United States,50 - 59 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,40.7,39.3,42.2,39.3 - 42.2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5149,"Delayed or Did Not Get Care, Last 4 Weeks",By State,Virginia,Virginia,3,24,Feb 3 - Feb 15,2021-02-03,2021-02-15,34.7,31.0,38.5,31.0 - 38.5,32.5-35.0
5150,"Delayed or Did Not Get Care, Last 4 Weeks",By State,Washington,Washington,3,24,Feb 3 - Feb 15,2021-02-03,2021-02-15,39.1,35.8,42.4,35.8 - 42.4,35.1-48.2
5151,"Delayed or Did Not Get Care, Last 4 Weeks",By State,West Virginia,West Virginia,3,24,Feb 3 - Feb 15,2021-02-03,2021-02-15,40.6,35.1,46.2,35.1 - 46.2,35.1-48.2
5152,"Delayed or Did Not Get Care, Last 4 Weeks",By State,Wisconsin,Wisconsin,3,24,Feb 3 - Feb 15,2021-02-03,2021-02-15,26.8,23.3,30.6,23.3 - 30.6,21.7-29.5


In [27]:
# Column selection based on the notes above

df = df[~df.Subgroup.isin(['With disability','Without disability'])]
df = df[df['Indicator'] == 'Delayed or Did Not Get Care, Last 4 Weeks']
df = df[['Subgroup', 'Phase', 'Time Period', 'Time Period Start Date', 'Time Period End Date', 'Value', 'Low CI', 'High CI']]

In [28]:
# Saving the resulting dataframe

df.to_csv(f'DATA/DATAFRAME_{month}_accessToCare.csv',index=False)

### June Data
The second round of data was collected in June so that our model created with the March data could be checked. To do this the June data must be explored to ensure it has the same information and cleaned so that it is in the same format as the March data. 

In [29]:
# Reading in the data and converting the date infomration to datetime format

month = 'JUN'
df = pd.read_csv(f"DATA/{month} - Indicators_of_Reduced_Access_to_Care_Due_to_the_Coronavirus_Pandemic_During_Last_4_Weeks.csv")
df.loc[:,'Time Period Start Date'] = df['Time Period Start Date'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%m/%d/%Y').date()) 
df.loc[:,'Time Period End Date'] = df['Time Period End Date'].apply(lambda x: datetime.datetime.strptime(x[0:10], '%m/%d/%Y').date()) 
df.head()

,Indicator,Group,State,Subgroup,Phase,Time Period,Time Period Label,Time Period Start Date,Time Period End Date,Value,Low CI,High CI,Confidence Interval,Quartile Range
0,"Delayed Medical Care, Last 4 Weeks",National Estimate,United States,United States,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,38.7,38.1,39.4,38.1 - 39.4,NaN
1,"Delayed Medical Care, Last 4 Weeks",By Age,United States,18 - 29 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,34.8,32.5,37.2,32.5 - 37.2,NaN
2,"Delayed Medical Care, Last 4 Weeks",By Age,United States,30 - 39 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,37.3,35.6,39.0,35.6 - 39.0,NaN
3,"Delayed Medical Care, Last 4 Weeks",By Age,United States,40 - 49 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,40.3,39.0,41.7,39.0 - 41.7,NaN
4,"Delayed Medical Care, Last 4 Weeks",By Age,United States,50 - 59 years,1,1,Apr 23 - May 5,2020-04-23,2020-05-05,40.7,39.3,42.2,39.3 - 42.2,NaN


In [30]:
# Looking at the dataframe info we see the same problem as before, missing values in our numeric columns as well as a problem with the Quartile Range

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7118 entries, 0 to 7117
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Indicator               7118 non-null   object 
 1   Group                   7118 non-null   object 
 2   State                   7118 non-null   object 
 3   Subgroup                7118 non-null   object 
 4   Phase                   7118 non-null   object 
 5   Time Period             7118 non-null   int64  
 6   Time Period Label       7118 non-null   object 
 7   Time Period Start Date  7118 non-null   object 
 8   Time Period End Date    7118 non-null   object 
 9   Value                   6966 non-null   float64
 10  Low CI                  6966 non-null   float64
 11  High CI                 6966 non-null   float64
 12  Confidence Interval     6966 non-null   object 
 13  Quartile Range          5049 non-null   object 
dtypes: float64(3), int64(1), object(10)
memo

In [31]:
# The time period column appears the same as before, just with a few more datapoints

df['Time Period'].unique()

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33],
      dtype=int64)

In [32]:
# The Indicator column has the same information, once again we will only be keeping the 'Delayed or Did Not Get Care, Last 4 Weeks' values

df.Indicator.unique()

array(['Delayed Medical Care, Last 4 Weeks',
       'Did Not Get Needed Care, Last 4 Weeks',
       'Delayed or Did Not Get Care, Last 4 Weeks'], dtype=object)

In [33]:
# In subgroups we see a few extra values, With and Without disability have been added, for our analysis we will be removing them

df.Subgroup.unique()

array(['United States', '18 - 29 years', '30 - 39 years', '40 - 49 years',
       '50 - 59 years', '60 - 69 years', '70 - 79 years',
       '80 years and above', 'Male', 'Female', 'Hispanic or Latino',
       'Non-Hispanic White, single race',
       'Non-Hispanic Black, single race',
       'Non-Hispanic Asian, single race',
       'Non-Hispanic, other races and multiple races',
       'Less than a high school diploma', 'High school diploma or GED',
       "Some college/Associate's degree", "Bachelor's degree or higher",
       'Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina

In [34]:
# Removing With and Withouth disability from the subgroup column

df = df[(df.Subgroup != 'With disability') & (df.Subgroup != 'Without disability')]

In [35]:
# The state column appears to have the same values

df.State.unique()

array(['United States', 'Alabama', 'Alaska', 'Arizona', 'Arkansas',
       'California', 'Colorado', 'Connecticut', 'Delaware',
       'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

In [36]:
# Onces again the United States value has the national average

df[df.State == 'United States'].Subgroup.unique()

array(['United States', '18 - 29 years', '30 - 39 years', '40 - 49 years',
       '50 - 59 years', '60 - 69 years', '70 - 79 years',
       '80 years and above', 'Male', 'Female', 'Hispanic or Latino',
       'Non-Hispanic White, single race',
       'Non-Hispanic Black, single race',
       'Non-Hispanic Asian, single race',
       'Non-Hispanic, other races and multiple races',
       'Less than a high school diploma', 'High school diploma or GED',
       "Some college/Associate's degree", "Bachelor's degree or higher"],
      dtype=object)

In [37]:
# The individual state values still only have data for the entire state, it would have been nice to see a better breakdown of the data in each state
# as it stands the state column still isn't useful

df[df.State == 'Alabama'].Subgroup.unique()

array(['Alabama'], dtype=object)

In [38]:
# Checking the Phase column we encounter the same strange values we found before

df.Phase.unique()

array(['1', '-1', '2', '3 (Oct 28 � Dec 21)', '3 (Jan 6 � Mar 29)', '3.1'],
      dtype=object)

In [39]:
# All the missing values in the Value field are in Phase -1

df[df.Value.isnull()].Phase.unique()

array(['-1'], dtype=object)

In [40]:
# Removing the Phase -1 as before

df = df[df.Phase != '-1']

In [41]:
# Again we see the strage Phase values have dates and information associated with them so they will be kept

df[df.Phase == '3 (Oct 28 � Dec 21)']['Time Period Start Date'].unique()

array([datetime.date(2020, 10, 28), datetime.date(2020, 11, 11),
       datetime.date(2020, 11, 25), datetime.date(2020, 12, 9)],
      dtype=object)

In [42]:
# Again we see the strage Phase values have dates and information associated with them so they will be kept

df[df.Phase == '3 (Jan 6 � Mar 29)']['Time Period Start Date'].unique()

array([datetime.date(2021, 1, 6), datetime.date(2021, 1, 20),
       datetime.date(2021, 2, 3), datetime.date(2021, 2, 17),
       datetime.date(2021, 3, 3), datetime.date(2021, 3, 17)],
      dtype=object)

In [43]:
# This is a new one but it has values have dates and information associated with them so they will be kept

df[df.Phase == '3.1']['Time Period Start Date'].unique()

array([datetime.date(2021, 4, 14), datetime.date(2021, 4, 28),
       datetime.date(2021, 5, 12), datetime.date(2021, 5, 26),
       datetime.date(2021, 6, 9), datetime.date(2021, 6, 23)],
      dtype=object)

In [44]:
# Converting the different Phase values to be '3' for ease of use

df.loc[df.Phase == '3 (Oct 28 � Dec 21)', 'Phase'] = 3
df.loc[df.Phase == '3 (Jan 6 � Mar 29)', 'Phase'] = 3
df.loc[df.Phase == '3.1', 'Phase'] = 3

In [45]:
# Data is selected as before to make it easier to handle 

df = df[~df.Subgroup.isin(['With disability','Without disability'])]
df = df[df['Indicator'] == 'Delayed or Did Not Get Care, Last 4 Weeks']
df = df[['Subgroup', 'Phase', 'Time Period', 'Time Period Start Date', 'Time Period End Date', 'Value', 'Low CI', 'High CI']]

In [46]:
# Saving the resulting dataframe

df.to_csv(f'DATA/DATAFRAME_{month}_accessToCare.csv',index=False)